# Lesson 4: Implementing Pagination

```markdown
# Implementing Pagination

Welcome back! You've secured your application with middleware and learned how to handle data validation and errors. Now, it’s time to make your To-Do list more user-friendly by implementing **pagination**. This feature is crucial for providing a smooth experience when users have many tasks.

## What You'll Learn
In this section, you’ll learn how to implement pagination in your Django application. Pagination divides content into separate pages, making it easier for users to navigate through large sets of data.

### Key Points:
- **Paginating Data**: Retrieve a subset of To-Do tasks for each page using Django's ORM.
- **User-Friendly Navigation**: Allow users to easily browse tasks.
  
---

## Implementing Pagination in Django

We’ll begin by creating a view in `project/myapp/views.py` that handles pagination.

```python
from django.http import JsonResponse
from .models import Todo

def get_todos(request):
    if request.method == 'GET':
        page = request.GET.get('page', 1)
        page = int(page) if page.isdigit() else 1
        page = 1 if page < 1 else page

        page_size = 3
        start_index = (page - 1) * page_size
        end_index = start_index + page_size

        todos = Todo.objects.filter(user=request.user)[start_index:end_index]
        return JsonResponse({'todos': list(todos.values())})
```

### Breakdown:
- **Page Number**: We retrieve the `page` number from the query parameters, ensuring it's a positive integer.
- **Page Size**: We define the number of tasks to display per page (`page_size = 3`).
- **Index Calculation**: Start and end indices are calculated to slice the tasks.
- **Querying Tasks**: We filter the tasks for the logged-in user and return the paginated result as JSON. Using `.values()` helps serialize the tasks as dictionaries.

---

## Making Paginated Requests

Finally, we'll show how to make requests to the paginated endpoint and display tasks in a user-friendly way.

```python
import requests

URL = 'http://127.0.0.1:3000'
page = 1
headers = {'Authorization': 'Token abc123'}

while True:
    response = requests.get(URL + f'/todos/?page={page}', headers=headers)
    data = response.json()
    todos = data.get('todos')
    if not todos:
        break
    print(f"Page {page}: {todos}")
    page += 1
```

### Breakdown:
- We send requests to the `/todos/` endpoint, passing the page number in the query string.
- We print the tasks for each page, stopping once there are no more tasks to display.

---

## Why It Matters

### Enhanced User Experience:
Pagination breaks down a large list of tasks into smaller, easily digestible chunks. This improves navigation and helps users find specific tasks more efficiently.

### Improved Performance:
Loading all data at once can slow down your application. Pagination helps load only the data required for each page, improving performance.

By mastering pagination, you’ll ensure your To-Do list application remains both efficient and user-friendly—even as the number of tasks grows.

---

Exciting, right? Now, let's practice by adding pagination to your To-Do list application! 🎉
```

## Implementing Pagination in Django

Great job so far! Now, let’s see pagination in action. Running the code will help you better understand how it works.

This exercise implements pagination for a To-Do list application, dividing tasks into pages. The get_todos view retrieves tasks based on the page number passed through query parameters. Note that the page size is set to 3 tasks per page, which means that the number of tasks displayed will be limited to 3.

Key concepts:

Paginating Data: Use Django's ORM to slice the queryset into pages. Calculate the start and end indices based on the page number and page size.

Using Query Parameters: Retrieve the page number from the GET parameters and use it to fetch the relevant subset of tasks.

Building the Request: Make HTTP requests to the paginated endpoint and handle the response by showing tasks page by page.

Simply hit the Run button to observe how we fetch data for each page of the To-Do list.

```python
from django.db import models
from django.contrib.auth.models import User

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task


from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt, ensure_csrf_cookie
from django.middleware.csrf import get_token
import json
from django.core.exceptions import ValidationError
from .models import Todo

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            csrf_token = 'abc123'
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': csrf_token})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)

@csrf_exempt
def get_todos(request):
    if request.method == 'GET':
        page = request.GET.get('page', 1)
        page = int(page) if page.isdigit() else 1
        page = 1 if page < 1 else page

        page_size = 3
        start_index = (page - 1) * page_size
        end_index = start_index + page_size

        todos = Todo.objects.filter(user=request.user)[start_index:end_index]
        return JsonResponse({'todos': list(todos.values())})


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')


headers = {'Authorization': f'Token {csrf_token}'}

if register_response.status_code == 201:
    # This is the first run, so put some data in the database
    todos = ["Buy groceries", "Walk the dog", "Do laundry", "Clean the house", "Study for exam", "Prepare presentation", "Call mom", "Pay bills", "Go to gym", "Read book"]
    for todo in todos:
        response = session.post(BASE_URL + "add-todo/", json={"task": todo}, headers=headers)

# Get paginated todos

page = 1
while True:
    response = session.get(BASE_URL + f"get-todos/?page={page}", headers=headers)
    data = response.json()
    todos = data.get('todos')
    if not todos:
        break
    print(f"Page {page}: {todos}")
    page += 1


from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
    path('get-todos/', views.get_todos, name='get_todos'),
]


```

This block of code demonstrates how pagination can be implemented in a Django-based To-Do list application. Below is an overview of its key components:

### 1. **Models (`Todo` model)**
The `Todo` model stores tasks for each user and enforces a validation rule on task length:

```python
from django.db import models
from django.contrib.auth.models import User

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task
```

### 2. **Authentication and CRUD Operations**
- **Register**: Allows users to register by providing a username, email, and password.
- **Login**: Users can log in, receive a CSRF token, and authenticate future requests.
- **Logout**: Logs the user out by clearing the session.
- **Add Todo**: Adds a new To-Do task for the authenticated user.
  
These operations are managed in the following views:

```python
from django.http import JsonResponse
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
import json
from django.core.exceptions import ValidationError
from .models import Todo

@csrf_exempt
def register(request):
    # User registration logic

@csrf_exempt
def user_login(request):
    # User login logic

@csrf_exempt
def user_logout(request):
    # User logout logic

@csrf_exempt
def add_todo(request):
    # Add new todo logic
```

### 3. **Pagination Logic**
The `get_todos` view implements pagination by fetching tasks based on the current page number, with 3 tasks displayed per page:

```python
@csrf_exempt
def get_todos(request):
    if request.method == 'GET':
        page = request.GET.get('page', 1)
        page = int(page) if page.isdigit() else 1
        page_size = 3
        start_index = (page - 1) * page_size
        end_index = start_index + page_size

        todos = Todo.objects.filter(user=request.user)[start_index:end_index]
        return JsonResponse({'todos': list(todos.values())})
```

### 4. **AuthMiddleware**
Middleware checks if the current route requires authorization and ensures that an authenticated user is making requests to protected routes:

```python
from django.urls import resolve
from django.http import JsonResponse

class AuthMiddleware:
    def __call__(self, request):
        # Middleware logic to verify token
```

### 5. **Testing Pagination with Requests**
In this section, we programmatically simulate the registration, login, and pagination of To-Do tasks using the `requests` library:

```python
import requests

BASE_URL = "http://127.0.0.1:3000/"

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register, login, and fetch paginated tasks
```

### 6. **URL Patterns**
URLs are mapped to their corresponding views:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
    path('get-todos/', views.get_todos, name='get_todos'),
]
```

### Conclusion
Running this code simulates how users can register, log in, add tasks, and retrieve paginated results. Pagination improves both usability and performance, especially when dealing with large datasets.

## Change Pagination to Show More Tasks

Great job so far! Now, let's modify the pagination to display 5 tasks per page instead of 3.

Update the get_todos function to show 5 tasks per page.

```python
from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token
from django.core.exceptions import ValidationError
import json
from .models import Todo

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)

def get_todos(request):
    # TODO: Modify the pagination logic to show 5 tasks per page
    if request.method == 'GET':
        page = request.GET.get('page', 1)
        page = int(page) if page.isdigit() else 1
        page = 1 if page < 1 else page

        page_size = 3
        start_index = (page - 1) * page_size
        end_index = start_index + page_size

        todos = Todo.objects.filter(user=request.user)[start_index:end_index]
        return JsonResponse({'todos': list(todos.values())})


from django.db import models
from django.contrib.auth.models import User
from django.core.exceptions import ValidationError

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')


headers = {'Authorization': f'Token {csrf_token}'}

if register_response.status_code == 201:
    # This is the first run, so put some data in the database
    todos = ["Buy groceries", "Walk the dog", "Do laundry", "Clean the house", "Study for exam", "Prepare presentation", "Call mom", "Pay bills", "Go to gym", "Read book"]
    for todo in todos:
        response = session.post(BASE_URL + "add-todo/", json={"task": todo}, headers=headers)

# Get paginated todos

page = 1
while True:
    response = session.get(BASE_URL + f"get-todos/?page={page}", headers=headers)
    data = response.json()
    todos = data.get('todos')
    if not todos:
        break
    print(f"Page {page}: {todos}")
    page += 1


from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
    path('get-todos/', views.get_todos, name='get_todos'),
]


```

To modify the pagination logic to display 5 tasks per page, we only need to adjust the `page_size` variable inside the `get_todos` view function.

Here’s the updated `get_todos` function:

```python
def get_todos(request):
    if request.method == 'GET':
        page = request.GET.get('page', 1)
        page = int(page) if page.isdigit() else 1
        page = 1 if page < 1 else page

        # Set page size to 5 tasks per page
        page_size = 5
        start_index = (page - 1) * page_size
        end_index = start_index + page_size

        todos = Todo.objects.filter(user=request.user)[start_index:end_index]
        return JsonResponse({'todos': list(todos.values())})
```

### Explanation of Changes:
- The `page_size` has been updated from `3` to `5`, so the application will now display 5 tasks per page.
  
With this change, when you run the pagination, the To-Do list will show 5 tasks per page instead of 3.

### Testing the Changes:
If you rerun the `while True` loop fetching paginated tasks from the To-Do list, you'll now see 5 tasks returned per page.

## Implementing Pagination Logic

Now that you have a basic understanding of pagination let's complete the get_todos function in the myapp/views.py file to return the todos based on the page number.

Follow the instructions in the file to complete the task.

```python
from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token
import json
from django.core.exceptions import ValidationError
from .models import Todo

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            csrf_token = 'abc123'
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': csrf_token})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
@login_required
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)

def get_todos(request):
    if request.method == 'GET':
        page = request.GET.get('page', 1)
        page = int(page) if page.isdigit() else 1
        page = 1 if page < 1 else page

        # TODO: Define the page_size = 3 and calculate the start and end indexes

        # Note, if page is 1, start_index should be 0 and end_index should be 3

        # TODO: Get the todos for the user (request.user) based on the start and end indexes

        todos = []
        return JsonResponse({'todos': list(todos.values())})


from django.db import models
from django.contrib.auth.models import User
from django.core.exceptions import ValidationError

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')


headers = {'Authorization': f'Token {csrf_token}'}

if register_response.status_code == 201:
    # This is the first run, so put some data in the database
    todos = ["Buy groceries", "Walk the dog", "Do laundry", "Clean the house", "Study for exam", "Prepare presentation", "Call mom", "Pay bills", "Go to gym", "Read book"]
    for todo in todos:
        response = session.post(BASE_URL + "add-todo/", json={"task": todo}, headers=headers)

# Get paginated todos

page = 1
while True:
    response = session.get(BASE_URL + f"get-todos/?page={page}", headers=headers)
    data = response.json()
    todos = data.get('todos')
    if not todos:
        break
    print(f"Page {page}: {todos}")
    page += 1


from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
    path('get-todos/', views.get_todos, name='get_todos'),
]


```

To implement the pagination logic in the `get_todos` function, follow these steps:

1. **Set Page Size**: Define a page size (e.g., `page_size = 3`) to control how many todos are returned per page.
2. **Calculate Start and End Indexes**: Use the page number to calculate the start and end indexes for slicing the list of todos.
3. **Retrieve Paginated Todos**: Query the todos for the authenticated user based on the calculated start and end indexes.

Here’s how you can modify the `get_todos` function:

### Updated `get_todos` Function

```python
@csrf_exempt
@login_required
def get_todos(request):
    if request.method == 'GET':
        # Get the current page number, default to 1 if not provided
        page = request.GET.get('page', 1)
        page = int(page) if page.isdigit() else 1
        page = 1 if page < 1 else page

        # Define the page size
        page_size = 3

        # Calculate start and end index for the given page
        start_index = (page - 1) * page_size
        end_index = start_index + page_size

        # Fetch todos for the logged-in user
        todos = Todo.objects.filter(user=request.user)[start_index:end_index]

        # Return the todos in JSON format
        return JsonResponse({'todos': list(todos.values())}, status=200)

    return JsonResponse({'message': 'Only GET method is allowed'}, status=405)
```

### Explanation:
1. **Page Parameter**: The `page` parameter is obtained from the query string (`request.GET.get('page', 1)`), with a fallback to 1 if not provided or invalid.
2. **Page Size**: A constant `page_size = 3` is defined, meaning each page will contain up to 3 todos.
3. **Start and End Index**: 
   - `start_index` is calculated as `(page - 1) * page_size`.
   - `end_index` is calculated as `start_index + page_size`.
4. **Querying Todos**: The todos for the authenticated user are sliced from the `start_index` to `end_index` using `Todo.objects.filter(user=request.user)[start_index:end_index]`.
5. **Return Response**: The resulting todos are converted into a list of dictionaries using `.values()` and returned in the response.

This should now paginate the todos properly based on the requested page number.

You've done well so far, completing various practice tasks on pagination.

Now, implement the full code for the get_todos view. Follow the instructions in the get_todos view to complete the implementation.

```python
from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token
from django.core.exceptions import ValidationError
import json
from .models import Todo

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            csrf_token = get_token(request)
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': csrf_token})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)

def get_todos(request):
    # TODO: Check if the request method is GET
        # TODO: Get the page number from the query parameters or set it to 1 if not provided or invalid

        # TODO: If the page number is not an integer, set it to 1

        # TODO: Check if the page number is less than 1 and set it to 1

        # TODO: Define the page size as 4 and calculate the start and end index for pagination

        # TODO: Retrieve the To-Do tasks based on the user and pagination indexes

        # TODO: Return the To-Do tasks as a JSON response with 'todos' key and list of tasks as the value

    return JsonResponse({'message': 'Invalid request'}, status=400)


from django.db import models
from django.contrib.auth.models import User
from django.core.exceptions import ValidationError

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.user.is_authenticated:
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')


headers = {'Authorization': f'Token {csrf_token}'}

if register_response.status_code == 201:
    # This is the first run, so put some data in the database
    todos = ["Buy groceries", "Walk the dog", "Do laundry", "Clean the house", "Study for exam", "Prepare presentation", "Call mom", "Pay bills", "Go to gym", "Read book"]
    for todo in todos:
        response = session.post(BASE_URL + "add-todo/", json={"task": todo}, headers=headers)

# Get paginated todos

page = 1
while True:
    response = session.get(BASE_URL + f"get-todos/?page={page}", headers=headers)
    data = response.json()
    todos = data.get('todos')
    if not todos:
        break
    print(f"Page {page}: {todos}")
    page += 1


from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
    path('get-todos/', views.get_todos, name='get_todos'),
]

```

Here’s the full implementation of the `get_todos` view based on the given instructions, including all necessary steps to handle pagination and return the todos for the current user.

### Completed `get_todos` Function

```python
@csrf_exempt
@login_required
def get_todos(request):
    if request.method == 'GET':
        # Get the page number from the query parameters, default to 1 if not provided or invalid
        page = request.GET.get('page', 1)
        try:
            page = int(page)
        except ValueError:
            page = 1

        # Ensure page is not less than 1
        if page < 1:
            page = 1

        # Define page size (number of todos per page)
        page_size = 4

        # Calculate start and end indexes for pagination
        start_index = (page - 1) * page_size
        end_index = start_index + page_size

        # Retrieve the todos for the logged-in user, based on pagination indexes
        todos = Todo.objects.filter(user=request.user)[start_index:end_index]

        # Return the todos in a JSON response
        return JsonResponse({'todos': list(todos.values())}, status=200)

    return JsonResponse({'message': 'Only GET method is allowed'}, status=405)
```

### Explanation:
1. **Request Method Check**: The function starts by checking if the request method is `GET`. If not, it returns a 405 error.
2. **Page Number Handling**:
   - It attempts to get the `page` from the query parameters. If it’s not present or invalid, it defaults to 1.
   - A `try-except` block ensures that the page value is an integer, and defaults to page 1 if conversion fails.
   - The page number is validated to be at least 1.
3. **Page Size**: The `page_size` is set to 4, which means 4 todos are returned per page.
4. **Start and End Indexes**: The `start_index` and `end_index` are calculated based on the current page.
5. **Querying Todos**: The todos are fetched for the authenticated user using the `filter` method, and sliced according to the start and end indexes.
6. **Returning Todos**: The todos are serialized into a list of dictionaries using `.values()` and returned in the JSON response.

### Example JSON Response
```json
{
    "todos": [
        {"id": 1, "task": "Buy groceries", "completed": false, "user_id": 2},
        {"id": 2, "task": "Walk the dog", "completed": false, "user_id": 2},
        {"id": 3, "task": "Do laundry", "completed": true, "user_id": 2},
        {"id": 4, "task": "Clean the house", "completed": false, "user_id": 2}
    ]
}
```

With this implementation, the todos are paginated, and you can get different sets of todos by providing the page number in the query string, e.g., `get-todos/?page=2`.